In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Loading the MNIST dataset with transforms
train_dataset = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)
test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [10]:
# DataLoader for batching
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

In [11]:
# Define the neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 512)  # Fully connected layer 1
        self.fc2 = nn.Linear(512, 10)       # Fully connected layer 2 (output)

    def forward(self, x):
        x = self.flatten(x)            # Flatten the input
        x = torch.relu(self.fc1(x))    # ReLU activation applied after first fully connected layer
        x = self.fc2(x)                # Output layer
        return x

In [12]:
# Initialize model, loss function, and optimizer
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Train the model
def train_model(train_loader):
    model.train()  # Set model to training mode
    for epoch in range(5):  # Training for 5 epochs
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f'Epoch [{epoch+1}/5], Loss: {running_loss/len(train_loader):.4f}, '
              f'Accuracy: {100 * correct / total:.2f}%')


In [14]:
# Test the model
def evaluate_model(test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')


In [15]:
# Run the training and evaluation
train_model(train_loader)
evaluate_model(test_loader)

Epoch [1/5], Loss: 0.3177, Accuracy: 91.31%
Epoch [2/5], Loss: 0.1302, Accuracy: 96.18%
Epoch [3/5], Loss: 0.0838, Accuracy: 97.57%
Epoch [4/5], Loss: 0.0609, Accuracy: 98.17%
Epoch [5/5], Loss: 0.0449, Accuracy: 98.70%
Test Accuracy: 97.85%
